# Capstone Project - Battle of the Neighborhood
This notebook has been created for the Coursera capstone project - **Toronto's battle of the Neighborhood**

#### STEP1:
* Use BeautifulSoup (bs4) to scrape the data from the source Wiki url
* Use get to fetch the contents of the URL, and format as xml
* The data we need is in the form of a table in the source (wikitable sortable)
* Create an empty dataframe, with 3 columns 'Postcode', 'Borough', 'Neighbourhood'
* Parse the xml source (looping through table data td and table row tr) and populating the dataframe with the parsed data
* Print the total rows in the dataframe using shape function
* Print the top 5 rows of the dataframe using head function

In [19]:
# Imports required to support scraping
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Data source for the scrape is the wiki url
WIKI_URL="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Pull data from the source url in xml format
req = requests.get(WIKI_URL)
soup = BeautifulSoup(req.content, 'lxml')

# Scrape the table data from the Wiki source required for this project
tables = soup.find("table", class_="wikitable sortable")

# Create an empty dataframe with the 3 columns matching the data source
col_names =  ['Postalcode', 'Borough', 'Neighbourhood']
df  = pd.DataFrame(columns = col_names)

# Parse the sortable table data from source, search for tr and td tags, and load the table data into the pandas dataframe
for tr in tables.find_all('tr'):
   tds = tr.find_all('td')
   if not tds:
      continue
   Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:3]]
   df.loc[len(df)] = [Postcode, Borough, Neighbourhood]

# Check the total rows in the dataframe
df.shape

(288, 3)

In [20]:
# Print top 5 rows of the dataframe to verify
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### STEP2:
* Remove all rows that does not have a Borough assigned
* Print the total rows in the dataframe using shape function
* Print the top 5 rows of the dataframe using head function

In [21]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df1 = df[df.Borough != 'Not assigned']
df1.reset_index(drop=True, inplace=True)

# Check the total rows in the dataframe (should be fewer rows)
df1.shape

(211, 3)

In [22]:
# Print top 5 rows of the dataframe to verify
df1.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### STEP3:
* Combine neighbourhood values as comma separated for same postcode and borough
* Print the total rows in the dataframe using shape function
* Print the top 5 rows of the dataframe using head function

In [23]:
# Combine neighbourhood values as comma separated for same postcode and borough
df2 = df1.groupby(['Postalcode','Borough'])['Neighbourhood'].agg(lambda x: ', '.join(set(x))).reset_index()

# Check the total rows in the dataframe (should be fewer rows)
df2.shape

(103, 3)

In [24]:
df2.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### STEP4:
* Set Neighbourhood as Borough for those Neighbourhoods which have a value of Not assigned in the Wiki source table
* Print the total rows in the dataframe using shape function
* Print the top 5 rows of the dataframe using head function

In [25]:
# Set Neighbourhood as Borough for those Neighbourhoods which have a value of Not assigned in the Wiki source table
df2.Neighbourhood.replace('Not assigned',df2.Borough,inplace=True)
df2.shape

(103, 3)

In [26]:
df2.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
# Print the shape of the final dataframe
df_final = df2
df_final.shape

(103, 3)

#### STEP5:
* Using geocode data from csv file to get latitude and longitude for all the postal codes, appending additional columns to the dataframe (Note: I could not get the geocoder to work, so commented out the geocoder code and used csv instead)
* Look up by postal code and populate the latitude and longitude in the original dataframe for all records
* Print the total rows in the dataframe using shape function
* Print the top 5 rows of the dataframe using head function

In [31]:
# !pip install geocoder
# import geocoder # import geocoder
# iterate over rows with iterrows()
# for index, row in df_final.iterrows():
#   # initialize your variable to None
#   lat_lng_coords = None
#   # loop until you get the coordinates
#   while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format(row.Postalcode))
#     lat_lng_coords = g.latlng
#   row.Latitude = lat_lng_coords[0]
#   row.Longitude = lat_lng_coords[1]
# df_final.head()

In [28]:
# Read the geospatial data csv file (provided in the project instruction) and load it into a new dataframe
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
# Loop up by postal code and populate latitude and longitude in the main dataframe
df_final['Latitude'] = df_final['Postalcode'].map(df_geo.set_index('Postal Code')['Latitude'].to_dict())
df_final['Longitude'] = df_final['Postalcode'].map(df_geo.set_index('Postal Code')['Longitude'].to_dict())
df_final.shape

(103, 5)

In [33]:
df_final.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
